In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import ssl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('../')
from fun.models import *
sns.set_style("whitegrid")

from tqdm import tqdm
import pandas as pd
torch.manual_seed(42)
np.random.seed(42)
import random
random.seed(42)
from torch.utils.data import Dataset, DataLoader

import os
import glob


Indeces of images

In [ ]:

test_set_xd = glob.glob("../data/test/*")
test_set_os = sorted([file.split(".")[0] for file in os.listdir("../data/test/")], key=len)


## Loading Data

In [ ]:
class TestDataset(Dataset):
    """Test Dataset from Kaggle"""

    def __init__(self, indeces, root_dir, transform=None):
        """
        Args:
            list (list): list of indeces.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.indeces = indeces
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.indeces)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name =  os.path.join(os.path.join(self.root_dir,
                                self.indeces[idx]) + ".png")
        image = io.imread(img_name)
        
        sample = image

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
test_ds = TestDataset(
    indeces=test_set_os,
    root_dir='../data/test/',
    transform=transforms.ToTensor()
    )

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


def test(dataloader, model):
    pred_lab = []

    model.eval()
 
    with torch.no_grad():
        for X in tqdm(dataloader):
            X = X.to(device)
            pred = model(X) 
            pred= pred.argmax(1)           
            
            pred_lab.append(pred)

    
    return pred_lab
    

In [ ]:
# from google.colab import drive
# drive.mount('drive')

### Load model to test

In [ ]:
model = Resnet18_2() 
model.load_state_dict(torch.load(r"C:\Users\aleks\Desktop\DS\SEM_2\DL\Project1\DeepLearning\models\augm_resnet2_bs8itd\model.pt"))
model.to(device)

In [ ]:

learning_rate = 1e-3
batch_size = 8
step_size = 5
gamma = 0.5
weight_decay=0.01


test_dataloader = DataLoader(
      test_ds, 
      batch_size=batch_size,
      shuffle=False
      )

pred_labels = test(test_dataloader, model)
    

In [ ]:
pre = np.array([ten.cpu().numpy() for ten in pred_labels])
all_labels = pre.flatten()
labels_map = {
    0:'plane', 
    1:'car', 
    2:'bird', 
    3:'cat',
    4:'deer', 
    5:'dog',
    6:'frog',
    7:'horse', 
    8:'ship', 
    9:'truck'
}

labels = pd.DataFrame(all_labels, columns = ["label_nr"])
labels.reset_index(inplace=True)
labels["labels"] = labels["label_nr"].map(labels_map)
labels.drop("label_nr", axis = 1, inplace = True)
labels.rename(columns = {"index" : "id", "labels": "label"}, inplace=True)
#labels.to_csv("../data/results/labels/Resnet18_2_augm/labels.csv", index= None)


In [ ]:
# torch.save(net.state_dict(), r"C:\Users\aleks\Desktop\DS\SEM_2\DL\Project1\DeepLearning\models\augm_resnet2_bs8itd\model.pt")